## Normal-cont. HDG tricks

We now turn over to the $H(\operatorname{div})$-HDG Stokes discretization:

* We will first a discuss how to apply the facet `dof` reduction 
* Afterwards, we will explain a relaxed $H(\operatorname{div})$-conforming discretizations to obtain a reduction of facet `dof`s also for the normal-continuity `dof`s 
* Finally, we will exploit knowledge about the pointwise divergence of the discrete solution (it's zero) to reduce the local `FESpace`s


### Projected jumps for the `TangentialFacetFESpace`

For the $H(\operatorname{div})$-HDG Stokes discretization we can apply the "projected jumps" modification as well, simply adding the same frags and the compression on the `TangentialFacetFESpace`. We will apply it below, after introducing further tuning tricks.


```{note}
Making the highest order `dof`s discontinuous is implemented for the facet spaces 
`FacetFESpace`, `TangentialFacetFESpace` and `NormalFacetFESpace` through the `highest_order_dc` flag.
Further, the flag `hide_highest_order_dc` is implemented for all these spaces.
```

### Relaxed $H(\operatorname{div})$-conforming `FESpace`s

After reducing the globally coupled tangential facet `dof`s we have degrees of freedom of order $k-1$ on the facets for the tangential component, but still order $k$ for the normal component (as we are in $H(\operatorname{div})$. In view of an optimal `dof`-reduction on the facets we can do even better:

By relaxing $H(\operatorname{div})$-conformity. Here, we again take the `dof`s for the highest order normal component and only for that we break up continuity, turning (again) one `INTERFACE_DOF` into two `LOCAL_DOF`s: 


<div style="display: flex; align-items: center; justify-content: center;">
    <img src="../_static/figures/hdiv_ho_cont.png" alt="sketch for hdiv cont" width="400px"/>
    <span style="font-size: 2em; margin: 0 20px;">→</span>
    <img src="../_static/figures/hdiv_ho_disc.png" alt="sketch for hdiv disc" width="400px"/>
</div>

The resulting velocity space (without the facet space) is

$$
V_h^T = \{ v \in [\mathbb{P}^k]^d(\mathcal{T}_h) \mid [\![\Pi^{k-1}_{\mathcal{T}_h} v \cdot n ]\!] = 0 \text{ on } \mathcal{F}_h\}
$$

where $[\![\cdot]\!]$ is the usual DG jump operator.

The relaxed $H(\operatorname{div})$-version of the `HDiv`-space in `NGSolve` is obtained with the `highest_order_dc` flag:
```
HDiv(mesh, ..., highest_order_dc=True)
```

```{note}
Breaking up the highest order continuity using the flag `highest_order_dc` is implemented in `NGSolve` also for the `FESpace`s: 
`TangentialFacetFESpace`, `NormalFacetFESpace`, `HDiv`, `HCurl`, `H1`, `HDivSurface`
```

We can map functions from the relaxed $H(\operatorname{div})$ space to the standard $H(\operatorname{div})$ space with a very simple average operator (that averages corresponding vector entries):

<div style="display: flex; align-items: center; justify-content: center;">
    <img src="../_static/figures/hdiv_ho_disc.png" alt="sketch for hdiv cont" width="400px"/>
    <span style="font-size: 2em; margin: 0 20px;">→</span>
    <img src="../_static/figures/hdiv_ho_cont.png" alt="sketch for hdiv disc" width="400px"/>
</div>

In `NGSolve` this is easily achieved with calling `Average` of the relaxed `HDiv` space:
```
V.Average(gfu.vec)
```

A few remarks on the application of the relaxed $H(\operatorname{div})$-conforming method:
1. When applying the relaxed $H(\operatorname{div})$-conforming method the velocity solution will in general not be normal-continuous (and thus solenoidal) anymore. With the `Average`-operator a fully $H(\operatorname{div})$-conforming velocity can be postprocessed. 
2. However, to also get "pressure robustness" for the Stokes problem the test function of the r.h.s. linear form needs to apply that average operator as well which is achieved with an application of the `Average`-operator (a.k.a. reconstruction operator) on the r.h.s. vector.
3. The relaxed $H(\operatorname{div})$-conforming HDG method works nice for the Stokes method and with the `Average`-operator preserves essentially all properties of the $H(\operatorname{div})$-conforming HDG method. However, for Navier-Stokes problems things are a bit more careful (as mass matrix and convection term would in general require the reconstruction operator as well which is algorithmically inconvenient)

```{note}
For more details on relaxed $H(\operatorname{div})$-conforming (H)DG methods see 
"P.L. Lederer, C. Lehrenfeld and J. Schöberl, Hybrid Discontinuous {Galerkin} methods with relaxed {H(div)}-conformity for incompressible flows. Part I, SINUM, 2018. https://doi.org/10.1137/17M1138078" and "P.L. Lederer, C. Lehrenfeld and J. Schöberl, Hybrid Discontinuous {Galerkin} methods with relaxed {H(div)}-conformity for incompressible flows. Part II, ESAIM: M2AN, 2019. https://doi.org/10.1051/m2an/2018054"

```

### Highest order divergence-free spaces

The `HDiv` space in `NGSolve` has a special construction of the hierarchical bases that allows as to reduce to all basis functions that have a divergence that is orthonal to the constants. For that the flag `hodivfree=True` can be provided to the `FESpace`.

When applied in the Stokes system, the pressure (as a Lagrange multiplier for the div-constraint), has to be reduced to piecewise constants in this case. 

```{note}
We note that with the `hodivfree`-flag only element-bubbles are removed in the velocity and the pressure space, i.e. we only achieve a reduction of  `LOCAL_DOF`s not `INTERFACE_DOF`s with this approach.
```

### Application to the Stokes discretization

We take the same geometry as in the previous example:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

In [ ]:
def solve_lin_system(a,f,gfu,inverse=""):
    aSinv = a.mat.Inverse(gfu.space.FreeDofs(coupling=a.condense),inverse=inverse)
    if a.condense:
        ainv = ((IdentityMatrix(a.mat.height) + a.harmonic_extension) @ (aSinv + a.inner_solve) @ (IdentityMatrix(a.mat.height) + a.harmonic_extension_trans))        
    else:
        ainv = aSinv
    rhs = f.vec.CreateVector()
    rhs.data = f.vec - a.mat * gfu.vec
    gfu.vec.data += ainv * rhs

In [ ]:
cyl = Cylinder((0,0,0), Z, h=0.5,r=1)
cyl.faces[0].name="side"
cyl.faces[1].name="top"
cyl.faces[2].name="bottom"
mesh = Mesh(OCCGeometry(cyl).GenerateMesh(maxh=0.3)).Curve(3)
Draw(mesh);

We consider **eight** different combination of spaces with three binary decisions:
* reduction of the tangential facet space to one order less (or not)
* reduction to a relaxed H(div) space with normal continuity only of one degree less (or not)
* reduction of interior $H(div)$ bubbles w.r.t. to the divergence (`hodivfree`)  (or not)



In [ ]:
order = 3

VT1 = HDiv(mesh,order=order, dirichlet="top|bottom|side")
VT2 = HDiv(mesh,order=order, dirichlet="top|bottom|side", hodivfree = True )
VT3 = HDiv(mesh,order=order, dirichlet="top|bottom|side", highest_order_dc=True)
VT4 = HDiv(mesh,order=order, dirichlet="top|bottom|side", hodivfree = True, highest_order_dc=True)
Vhat1 = TangentialFacetFESpace(mesh,order=order, dirichlet="bottom|top")
Vhat2 = Compress(TangentialFacetFESpace(mesh,order=order, dirichlet="bottom|top", highest_order_dc=True, hide_highest_order_dc=True))
Q1 = L2(mesh,order=order-1, lowest_order_wb=True)
Q2 = L2(mesh,order=0, lowest_order_wb=True)
N = NumberSpace(mesh)
fes_params= [(VT1,Vhat1,Q1,"std"), \
             (VT1,Vhat2,Q1,"t. red."), \
             (VT3,Vhat1,Q1,"n. red."), \
             (VT3,Vhat2,Q1,"n.+t. red."), \
             (VT2,Vhat1,Q2,"            hodf"), \
             (VT2,Vhat2,Q2,"t. red.,    hodf"), \
             (VT4,Vhat1,Q2,"n. red.,    hodf"), \
             (VT4,Vhat2,Q2,"n.+t. red., hodf")]
labeled_fes = [(VT * Vhat * Q * N, name) for VT,Vhat,Q,name in fes_params]

We setup the HDG Stokes system:

In [ ]:
def setup_Stokes_HDG_system(fes, **opts):
    nu = 1
    gamma = 10
    yh, zh = fes.TnT()
    order = fes.components[0].globalorder
    (u, uhat, p, lam), (v, vhat, q, mu) = yh, zh
    uh, vh = (u, uhat), (v, vhat)
    tang = lambda u: u - (u*n)*n
    tjump = lambda uh: tang(uh[0]-uh[1]) 

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    K = BilinearForm(fes, **opts)
    K += (nu*InnerProduct(Grad(u), Grad(v)) + div(u)*q + div(v)*p + p * mu + q * lam) * dx
    K += nu * Grad(u) * n * tjump(vh) * dx(element_boundary = True)
    K += nu * Grad(v) * n * tjump(uh) * dx(element_boundary = True)
    K += nu  *gamma*order**2/h * InnerProduct ( tjump(vh),  tjump(uh) ) * dx(element_boundary = True)
    K.Assemble()

    r = sqrt(x**2+y**2)
    f = LinearForm(fes)
    f += 500*CF((0,0,r-0.5))*v*dx
    f.Assemble()

    fes.components[0].Average(f.vec)

    return K,f


For different options on how to eliminate `dof`s and for the different methods we compute some metrics of the solution of the resulting linear systems:
Effectively for every combination of methods we call:
```
a,f = setup_Stokes_HDG_system(fes, ...)
solve_lin_system(a,f, gfu=gfu)
```

In [ ]:
linsys_opts = [({"condense": False},"  none"), \
               ({"eliminate_hidden": True},"hidden"), \
               ({"condense": True}," local")]
import time
try:
  import pandas as pd
  from IPython.display import display, HTML
  html_output = True
except:
  html_output = False
data = []
with TaskManager():
  for opts, elims in linsys_opts:
    if not html_output:
      print(f"\n\n dof elimination: {elims:9} \n")
      print(f"                |           ndofs          | ncdofs |     nze  | timing (ass.+sol.)")
      print(f"----------------|--------------------------|--------|----------|-------------------")    
    for fes, label in labeled_fes:
      gfu = GridFunction(fes)
      gfu.components[1].Set(CF((y,-x,0)), definedon=mesh.Boundaries("bottom"))
      timer1 = time.time()
      a,f = setup_Stokes_HDG_system(fes, **opts)
      timer2 = time.time()
      solve_lin_system(a,f, gfu=gfu)
      timer3 = time.time()
      if html_output:
        data.append([elims,label,fes.ndof,fes.components[0].ndof,fes.components[1].ndof,
                     fes.components[2].ndof,fes.components[3].ndof,
                     sum(fes.FreeDofs(coupling=True)),
                     "{:.0f}K".format(a.mat.nze/1000),"{:5.3f}".format(timer3-timer1),
                     "{:5.3f}".format(timer2-timer1),"{:5.3f}".format(timer3-timer2)])
      else:
        print(f"{label:16}|",end="")
        print(f"{fes.ndof:5}({fes.components[0].ndof:5}+{fes.components[1].ndof:5}",end="")
        print(f"{fes.components[2].ndof:5}+{fes.components[3].ndof:1})",end=" ")
        print(f"|{sum(fes.FreeDofs(coupling=True)):7}", end=" ")
        print(f"|{a.mat.nze:9}", end=" ")
        print(f"| {timer3-timer1:4.2f} = {timer2-timer1:4.2f} + {timer3-timer2:4.2f}")
if not html_output:
  print(f"----------------|--------------------------|--------|----------|-------------------")
  print("")
else:
  display(HTML(pd.DataFrame(data, 
                            columns=["dof elim.", "FESpace", "ndofs", "nd0", "nd1", 
                                     "nd2", "nd3","ncdofs","nze","time","ass.",
                                     "sol."]).to_html(border=0,index=False)))                     

```{WARNING}
The combination of `tang. red` and no condensation (`condensed dofs: none`) will not yield reasonable numerical results, see discussion above. 
```

Let us check if the most reduced solution space (which yields the cheapest solution) is still able to solve the problem, by inspecting the solution:

In [ ]:
N = 15
points = [ (sin(2*pi*k/N)*i/N, cos(2*pi*k/N)*i/N, j/N)   for i in range(1,N) for j in range(1,N) for k in range(0,N)]
fl = gfu.components[0]._BuildFieldLines(mesh, points, num_fieldlines=N**3//20, randomized=True, length=1)
ea = { "euler_angles" : (-40, 0, 0) }
Draw(gfu.components[0], mesh,  "X", draw_vol=False, draw_surf=True, objects=[fl], \
     min=0, max=1, autoscale=False, settings={"Objects": {"Surface": False, "Wireframe":False}}, **ea);